# Linear Regression Type II

In [2]:
pip install pylr2

Note: you may need to restart the kernel to use updated packages.


In [3]:
from pylr2  import regress2

In [4]:
import pandas as pd

**S Bell Notes:**

You've installed a package above called **pylr2** and loaded independently its ___regress2___ method, so when you define the regress2 method again below, you will overwrite your initial import.  **TL;DR** you only need to import above or define below but you don't need to do both.  Now if your code below was to better understand the imported library, then ignore my comments.

In [12]:
import statsmodels.api as sm
import numpy as np


def regress2(_x, _y, _method_type_1 = "ordinary least square",
             _method_type_2 = "reduced major axis",
             _weight_x = [], _weight_y = [], _need_intercept = True):
    # Regression Type II based on statsmodels
    # Type II regressions are recommended if there is variability on both x and y
    # It's computing the linear regression type I for (x,y) and (y,x)
    # and then average relationship with one of the type II methods
    #
    # INPUT:
    #   _x <np.array>
    #   _y <np.array>
    #   _method_type_1 <str> method to use for regression type I:
    #     ordinary least square or OLS <default>
    #     weighted least square or WLS
    #     robust linear model or RLM
    #   _method_type_2 <str> method to use for regression type II:
    #     major axis
    #     reduced major axis <default> (also known as geometric mean)
    #     arithmetic mean
    #   _need_intercept <bool>
    #     True <default> add a constant to relation (y = a x + b)
    #     False force relation by 0 (y = a x)
    #   _weight_x <np.array> containing the weigth of x
    #   _weigth_y <np.array> containing the weigth of y
    #
    # OUTPUT:
    #   slope
    #   intercept
    #   r
    #   std_slope
    #   std_intercept
    #   predict
    #
    # REQUIRE:
    #   numpy
    #   statsmodels
    #
    # The code is based on the matlab function of MBARI.
    # AUTHOR: Nils Haentjens
    # REFERENCE: https://www.mbari.org/products/research-software/matlab-scripts-linear-regressions/

    # Check input
    if _method_type_2 != "reduced major axis" and _method_type_1 != "ordinary least square":
        raise ValueError("'" + _method_type_2 + "' only supports '" + _method_type_1 + "' method as type 1.")

    # Set x, y depending on intercept requirement
    if _need_intercept:
        x_intercept = sm.add_constant(_x)
        y_intercept = sm.add_constant(_y)

    # Compute Regression Type I (if type II requires it)
    if (_method_type_2 == "reduced major axis" or
        _method_type_2 == "geometric mean"):
        if _method_type_1 == "OLS" or _method_type_1 == "ordinary least square":
            if _need_intercept:
                [intercept_a, slope_a] = sm.OLS(_y, x_intercept).fit().params
                [intercept_b, slope_b] = sm.OLS(_x, y_intercept).fit().params
            else:
                slope_a = sm.OLS(_y, _x).fit().params
                slope_b = sm.OLS(_x, _y).fit().params
        elif _method_type_1 == "WLS" or _method_type_1 == "weighted least square":
            if _need_intercept:
                [intercept_a, slope_a] = sm.WLS(
                    _y, x_intercept, weights=1. / _weight_y).fit().params
                [intercept_b, slope_b] = sm.WLS(
                    _x, y_intercept, weights=1. / _weight_x).fit().params
            else:
                slope_a = sm.WLS(_y, _x, weights=1. / _weight_y).fit().params
                slope_b = sm.WLS(_x, _y, weights=1. / _weight_x).fit().params
        elif _method_type_1 == "RLM" or _method_type_1 == "robust linear model":
            if _need_intercept:
                [intercept_a, slope_a] = sm.RLM(_y, x_intercept).fit().params
                [intercept_b, slope_b] = sm.RLM(_x, y_intercept).fit().params
            else:
                slope_a = sm.RLM(_y, _x).fit().params
                slope_b = sm.RLM(_x, _y).fit().params
        else:
            raise ValueError("Invalid literal for _method_type_1: " + _method_type_1)

    # Compute Regression Type II
    if (_method_type_2 == "reduced major axis" or
        _method_type_2 == "geometric mean"):
        # Transpose coefficients
        if _need_intercept:
            intercept_b = -intercept_b / slope_b
        slope_b = 1 / slope_b
        # Check if correlated in same direction
        if np.sign(slope_a) != np.sign(slope_b):
            raise RuntimeError('Type I regressions of opposite sign.')
        # Compute Reduced Major Axis Slope
        slope = np.sign(slope_a) * np.sqrt(slope_a * slope_b)
        if _need_intercept:
            # Compute Intercept (use mean for least square)
            if _method_type_1 == "OLS" or _method_type_1 == "ordinary least square":
                intercept = np.mean(_y) - slope * np.mean(_x)
            else:
                intercept = np.median(_y) - slope * np.median(_x)
        else:
            intercept = 0
        # Compute r
        r = np.sign(slope_a) * np.sqrt(slope_a / slope_b)
        # Compute predicted values
        predict = slope * _x + intercept
        # Compute standard deviation of the slope and the intercept
        n = len(_x)
        diff = _y - predict
        Sx2 = np.sum(np.multiply(_x, _x))
        den = n * Sx2 - np.sum(_x) ** 2
        s2 = np.sum(np.multiply(diff, diff)) / (n - 2)
        std_slope = np.sqrt(n * s2 / den)
        if _need_intercept:
            std_intercept = np.sqrt(Sx2 * s2 / den)
        else:
            std_intercept = 0
    elif (_method_type_2 == "Pearson's major axis" or
          _method_type_2 == "major axis"):
        if not _need_intercept:
            raise ValueError("Invalid value for _need_intercept: " + str(_need_intercept))
        xm = np.mean(_x)
        ym = np.mean(_y)
        xp = _x - xm
        yp = _y - ym
        sumx2 = np.sum(np.multiply(xp, xp))
        sumy2 = np.sum(np.multiply(yp, yp))
        sumxy = np.sum(np.multiply(xp, yp))
        slope = ((sumy2 - sumx2 + np.sqrt((sumy2 - sumx2)**2 + 4 * sumxy**2)) /
                 (2 * sumxy))
        intercept = ym - slope * xm
        # Compute r
        r = sumxy / np.sqrt(sumx2 * sumy2)
        # Compute standard deviation of the slope and the intercept
        n = len(_x)
        std_slope = (slope / r) * np.sqrt((1 - r ** 2) / n)
        sigx = np.sqrt(sumx2 / (n - 1))
        sigy = np.sqrt(sumy2 / (n - 1))
        std_i1 = (sigy - sigx * slope) ** 2
        std_i2 = (2 * sigx * sigy) + ((xm ** 2 * slope * (1 + r)) / r ** 2)
        std_intercept = np.sqrt((std_i1 + ((1 - r) * slope * std_i2)) / n)
        # Compute predicted values
        predict = slope * _x + intercept
    elif _method_type_2 == "arithmetic mean":
        if not _need_intercept:
            raise ValueError("Invalid value for _need_intercept: " + str(_need_intercept))
        n = len(_x)
        sg = np.floor(n / 2)
        # Sort x and y in order of x
        sorted_index = sorted(range(len(_x)), key=lambda i: _x[i])
        x_w = np.array([_x[i] for i in sorted_index])
        y_w = np.array([_y[i] for i in sorted_index])
        x1 = x_w[1:sg + 1]
        x2 = x_w[sg:n]
        y1 = y_w[1:sg + 1]
        y2 = y_w[sg:n]
        x1m = np.mean(x1)
        x2m = np.mean(x2)
        y1m = np.mean(y1)
        y2m = np.mean(y2)
        xm = (x1m + x2m) / 2
        ym = (y1m + y2m) / 2
        slope = (x2m - x1m) / (y2m - y1m)
        intercept = ym - xm * slope
        # r (to verify)
        r = []
        # Compute predicted values
        predict = slope * _x + intercept
        # Compute standard deviation of the slope and the intercept
        std_slope = []
        std_intercept = []

    # Return all that
    return {"slope": float(slope), "intercept": intercept, "r": r,
            "std_slope": std_slope, "std_intercept": std_intercept,
            "predict": predict}


if __name__ == '__main__':
    x = np.linspace(0, 10, 100)
    # Add random error on y
    e = np.random.normal(size=len(x))
    y = x + e
    results = regress2(x, y, _method_type_2="reduced major axis",
                       _need_intercept=False)

In [5]:
sd_df = pd.read_csv('data/1033.csv')
sd_df.sample()

,X,Y
13,1.307146,1.234206


In [6]:
import numpy as np #SBell note you needto import numpyto use np.* modules

In [8]:
#generate a dataset
x=np.linspace(0,10,100)
e=np.random.normal(size=len(x))
y=x+e
#compute regression type 2
results=regress2(x,y,_method_type_2="reduced major axis")



In [9]:
results #using the  generated dataset above with x and y created via numpy calls

{'slope': 1.0221012963889173,
 'intercept': -0.056509250680926826,
 'r': 0.9464232900516896,
 'std_slope': 0.03379744885659501,
 'std_intercept': 0.19562179302934607,
 'predict': array([-0.05650925,  0.0467333 ,  0.14997586,  0.25321841,  0.35646097,
         0.45970353,  0.56294608,  0.66618864,  0.76943119,  0.87267375,
         0.9759163 ,  1.07915886,  1.18240141,  1.28564397,  1.38888652,
         1.49212908,  1.59537163,  1.69861419,  1.80185674,  1.9050993 ,
         2.00834185,  2.11158441,  2.21482696,  2.31806952,  2.42131207,
         2.52455463,  2.62779718,  2.73103974,  2.83428229,  2.93752485,
         3.04076741,  3.14400996,  3.24725252,  3.35049507,  3.45373763,
         3.55698018,  3.66022274,  3.76346529,  3.86670785,  3.9699504 ,
         4.07319296,  4.17643551,  4.27967807,  4.38292062,  4.48616318,
         4.58940573,  4.69264829,  4.79589084,  4.8991334 ,  5.00237595,
         5.10561851,  5.20886106,  5.31210362,  5.41534617,  5.51858873,
         5.62183128

**S Bell Notes:**

you've imported your data into a pandas dataframe (thats great) called sd_df and each column is accessible via either `sd_df.X` or `sd_df['X']` and `sd_df.Y` or `sd_df['Y']` so if the regress2 code is pandas friendly, you ought to be able to just replace x, y with the mentioned methods (the different ways to access a column are important but not for this case)... so...

`results=regress2(sd_df.X,sd_df.Y,_method_type_2="reduced major axis")`
or
`results=regress2(sd_df['X'],sd_df['Y'],_method_type_2="reduced major axis")`

but this passes a pandas.series variable to the regress2 method (thats a level of additional complexity pandas puts on a variable) ... if for some reason the code is not able to use that, you will need to pass just the raw values of the pandas column as an array... you do this buy usint the .values method on the column pandas.series e.g. `sd_df.X.values` or `sd_df['X'].values` which will send a numpy array to the method.

In [11]:
#generate a dataset
x=sd_df.X
y=sd_df.Y
#compute regression type 2
results=regress2(x,y,_method_type_2="reduced major axis")
results

{'slope': 2.139694559470794,
 'intercept': 0.2960691510161908,
 'r': 0.5614386400735981,
 'std_slope': 0.24481868526525158,
 'std_intercept': 0.2161295110018857,
 'predict': 0     5.938483
 1     2.639748
 2     0.484641
 3     0.643326
 4     1.080860
         ...   
 64    1.209524
 65    1.875758
 66    1.113102
 67    1.676336
 68    1.998909
 Name: X, Length: 69, dtype: float64}

In [12]:
#generate a dataset
x=sd_df['X']
y=sd_df['Y']
#compute regression type 2
results=regress2(x,y,_method_type_2="reduced major axis")
results

{'slope': 2.139694559470794,
 'intercept': 0.2960691510161908,
 'r': 0.5614386400735981,
 'std_slope': 0.24481868526525158,
 'std_intercept': 0.2161295110018857,
 'predict': 0     5.938483
 1     2.639748
 2     0.484641
 3     0.643326
 4     1.080860
         ...   
 64    1.209524
 65    1.875758
 66    1.113102
 67    1.676336
 68    1.998909
 Name: X, Length: 69, dtype: float64}

In [13]:
#generate a dataset
x=sd_df['X'].values
y=sd_df['Y'].values
#compute regression type 2
results=regress2(x,y,_method_type_2="reduced major axis")
results

{'slope': 2.139694559470794,
 'intercept': 0.2960691510161908,
 'r': 0.5614386400735981,
 'std_slope': 0.24481868526525158,
 'std_intercept': 0.2161295110018857,
 'predict': array([5.93848333, 2.63974792, 0.48464093, 0.6433259 , 1.08085955,
        0.65054522, 0.59551056, 0.77890428, 4.58138486, 3.20719687,
        1.26676474, 0.82279967, 1.16387495, 3.09296198, 0.64324575,
        0.61414069, 0.66051459, 0.58820883, 0.62543354, 0.62921725,
        0.69038272, 0.65903241, 0.69740644, 1.14431852, 0.94766524,
        0.71448489, 0.8408004 , 0.90761291, 0.85213713, 0.83592962,
        0.77550641, 0.63652291, 0.72886139, 0.77082613, 0.62642188,
        0.66247666, 0.88788398, 1.0189293 , 0.53644728, 0.58308926,
        0.54486808, 0.69555773, 0.61827149, 1.01368979, 1.40781878,
        1.16888622, 2.52191325, 3.64101093, 3.59210326, 2.41904785,
        2.06471873, 2.88490668, 1.98684323, 2.48410456, 3.65919139,
        3.45818414, 3.30549972, 4.55486282, 4.46947822, 4.28955755,
        3.5